### Solution 1

In [ ]:
import google.generativeai as genai

genai.configure(api_key="<your key>")


def get_completion(prompt):
    """
    Get the completion for a given prompt using the specified model.
    Returns the answer with the highest score.
    """
    model = genai.GenerativeModel(
            "models/gemini-1.5-flash",
            system_instruction="You are a user.",
        )
    response = model.generate_content(prompt)
    return response.text


For the following text:
Needed a nice lamp for my bedroom, and this one had 
additional storage and not too high of a price point. 
Got it fast.  The string to our lamp broke during the 
transit and the company happily sent over a new one. 
Came within a few days as well. It was easy to put 
together.  I had a missing part, so I contacted their 
support and they very quickly got me the missing piece! 
Lumina seems to me to be a great company that cares 
about their customers and products!!

Validate if this author of this text is happy or not (answer in a single word: Positive/Negative)

In [ ]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Instruct the AI to write a the best unit test for the following code:
```
import requests

def get_user_data(user_id):
    url = f"https://api.example.com/users/{user_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()
```


In [ ]:
code = """
import requests

def get_user_data(user_id):
    url = f"https://api.example.com/users/{user_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()
"""

prompt = f"""
I have the following code: '''{code}'''. Write a unit test using pytest patch 
Give a solution using a decorator 
"""

print(get_completion(prompt))